# Proyecto Final del curso Data Engineering de Coderhouse
## Consumidor de API de Spotify

<p align="center">
  <img src="https://www.telia.se/.imaging/default/dam/telia-se/privat/Spotify-logo-Icons/Spotify_Icon_Green_NY_336x336/jcr:content.png" alt="Spotify Logo"/>
</p>

Mi nombre es Rodrigo Vega Gimenez. En la primera entrega estoy dejando esquematizado el proceso para consumir la API de Spotify, habiendo creado de 0 una app desde la misma aplicacion. La misma resume datos obtenidos de canciones usando una accion de get con una libreria llamada Spotipy.
Me guie por tutoriales enumerados a continuacion:

[How to retrieve data from Spotify](https://medium.com/@rafaelnduarte/how-to-retrieve-data-from-spotify-110c859ab304)

[Getting started - WebAPI Spotify](https://developer.spotify.com/documentation/web-api/tutorials/getting-started)

[Exploring the Spotify API in Python](https://stmorse.github.io/journal/spotify-api.html)

In [40]:
# Primero instalamos las librerias necesarias.
!pip install spotipy
!pip install wheel
!pip install pandas
!pip install psycopg2

In [41]:
# Despues obtenemos el client_id y el client_secret, almacenados en mi PC.
with open("C:/Users/rodri/Documents/keys/spotify_client_id.txt", 'r') as fid:
    client_id = fid.read()
print(client_id)

with open("C:/Users/rodri/Documents/keys/spotify_client_secret.txt", 'r') as fs:
    client_secret = fs.read()

3e2c784859ce4b97a9e449a44f619755


In [42]:
# Para realizar la conexion con Redshift es necesario obtener todos los parametros de acceso.
# La contraseña tambien es accesible mediante un archivo local en mi PC.
import psycopg2 as redshift_client
url = "data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
data_base = "data-engineer-database"
user = "rvega389_coderhouse"
with open("C:/Users/rodri/Documents/keys/redshift_password.txt",'r') as frp:
    redshift_pwd = frp.read()
try:
    connection = redshift_client.connect(
        host = url,
        dbname = data_base,
        user = user,
        password = redshift_pwd,
        port = '5439'
    )
    print("Connected to Redshift succesfully")
    
except Exception as e:
    print("It is not possible to connect to Redshift")
    print(e)

Connected to Redshift succesfully


In [43]:
# Seguimos con el log de las credenciales en Spotify.
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
spotify_client = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
# Creamos listas vacias para guardar los resultados.
# NOTA: Spotify limito los offset de 10k a 2k, por lo que decidi crear un loop cada 2k registros aumentando el offset de 50 en 50.
artist_names = []
artist_ids = []
track_names = []
track_ids = []
album_names = []
popularity = []
track_genres = []
track_years = []

for i in range(0,2000,50):
    track_results = spotify_client.search(q='year:2023', type='track', limit=50)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_ids.append(t['artists'][0]['id'])
        artist_names.append(t['artists'][0]['name'])
        # Sacamos las comillas del titulo de la cancion.
        track_names.append(t['name'].replace("'", ""))
        track_ids.append(t['id'])
        # Sacamos las comillas tambien del titulo del album.
        album_names.append(t['album']['name'].replace("'", ""))
        popularity.append(t['popularity'])
        track_genre = spotify_client.artist(artist_ids[i])['genres']
        # Separamos los generos por coma.
        track_genre = ', '.join(track_genre)
        track_genres.append(track_genre)
        track_years.append(t['album']['release_date'])

# Obtenemos el tamaño de uno, y chequeamos que sean 2000.
print('Numero de elementos en la lista de ids de canciones:', len(track_ids))

In [ ]:
import pandas as pd

# Generamos el dataframe con las listas.
df_tracks = pd.DataFrame({'ID Cancion': track_ids, 'Cancion': track_names, 'Artista': artist_names, 'ID Artista': artist_ids, 'Album': album_names, 'Popularidad': popularity, 'Genero': track_genres, 'Año': track_years})
# Obtenemos las dimensiones del dataframe.
print(f'Antes de borrar duplicados: {df_tracks.shape}')

# A partir de aqui, son acciones paleativas para mejorar el resultado mostrado.
# En el caso de que haya canciones duplicadas, borramos las mismas.
# Primero las agrupamos por artista y cancion.
grouped = df_tracks.groupby(['Artista','Cancion'], as_index = True).size()
grouped[grouped > 1].count()
# Posteriormente, se borran los duplicados.
df_tracks.drop_duplicates(subset = ['Artista','Cancion'], inplace = True)

# Chequeamos el nuevo tamaño ahora.
print(f'Despues de borrar duplicados: {df_tracks.shape}')

In [ ]:
# Otro aporte que encontre, de Tomigelo, sugiere que hay que implementar un check para validar que un track_id venga vacio, y tambien eliminarlo.
rows = []
batchsize = 100
no_track_id_counter = 0

for i in range(0,len(df_tracks['ID Cancion']),batchsize):
    batch = df_tracks['ID Cancion'][i:i+batchsize]
    feature_results = spotify_client.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            no_track_id_counter = no_track_id_counter + 1
        else:
            rows.append(t)
            
print('Numero de canciones sin soporte:', no_track_id_counter)
# En este caso, menos mal que no hay ninguna.

In [ ]:
# Finalmente, un par de validaciones mas.
# Si en el campo genero viene vacio, completar con Desconocido.
df_tracks['Genero'].fillna('Desconocido', inplace = True)
df_tracks.loc[df_tracks['Genero'] == '', 'Genero'] = 'Desconocido'
# Validar que la fecha venga en formato de fecha. 
df_tracks['Año'] = pd.to_datetime(df_tracks['Año'], format='%Y-%m-%d')
df_tracks.head()

In [ ]:
# Ahora vamos con todas las acciones sobre Redshift.
# Es necesario crear la tabla en el caso de que la misma no exista.
with connection.cursor() as cur:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS rvega389_coderhouse.canciones
        (
	    id VARCHAR(50) primary key
        ,cancion VARCHAR(255)
	    ,artista VARCHAR(255)   
	    ,artista_id VARCHAR(50)  
	    ,genero VARCHAR(300)   
	    ,album VARCHAR(100)   
	    ,popularidad INTEGER 
	    ,fecha_lanzamiento date   
        )
    """)
    connection.commit()

In [ ]:
# En primera instancia, borramos todos los elementos de la tabla para evitar duplicados e inconsistencias.
with connection.cursor() as cur:
  cur.execute("TRUNCATE TABLE canciones")
  count = cur.rowcount
    
# Si consultamos la tabla, la misma deberia estar vacia.
cur = connection.cursor()
cur.execute("SELECT * FROM canciones")
results = cur.fetchall()
results

In [ ]:
# Finalmente, insertamos el dataframe generado dentro de la base de datos.
from psycopg2.extras import execute_values as ejecutar
with connection.cursor() as cur:
    ejecutar(
        cur,
        '''
        INSERT INTO canciones (Id, Cancion, Artista, Artista_id, Album, Popularidad, Genero, Fecha_lanzamiento)
        VALUES %s
        ''',
        [tuple(row) for row in df_tracks.values],
        page_size=len(df_tracks)
    )
    connection.commit()

In [ ]:
# Si consultamos la tabla, la misma deberia tener los elementos recien almacenados.
cur = connection.cursor()
cur.execute("SELECT * FROM canciones")
results = cur.fetchall()
results

In [ ]:
# Y para cerrar, closeamos todas las conexiones.
cur.close()
connection.close()